In [49]:

from __future__ import print_function

import numpy as np
import warnings
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs


def identity_block(input_tensor, kernel_size, filters, stage, block):

    filters1, filters2, filters3 = filters
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3,name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3,name=bn_name_base + '2c')(x)
    x = Activation('relu')(x)
    
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):

    filters1, filters2, filters3 = filters
  
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization( axis=3,name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


In [26]:
def ResNet50(input_shape):
    inpt = Input(input_shape)
    x = ZeroPadding2D((3, 3))(inpt)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=3,name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = AveragePooling2D((2,2))(x)
    
    x = Flatten()(x)
    x = Dense(10, activation='softmax', name='res')(x)
    
    model = Model(inputs=inpt ,outputs=x, name='mnistmodel')
    
    return model

In [87]:
model = ResNet50([28,28,1])
model.compute_output_shape((42000,28,28,1))

(42000, 10)

In [74]:
df = pd.read_csv('train.csv')

In [75]:
lable = df['label'].values

In [76]:
label.shape

(42000,)

In [77]:

x = df.drop('label',axis=1).values

In [78]:
x.shape

(42000, 784)

In [79]:
datas = []
for i in range(x.shape[0]):
    datas.append(x[i,:].reshape(28,28,1))

In [80]:
results = np.array(datas)

In [81]:
results.shape

(42000, 28, 28, 1)

In [82]:
labels = []
with tf.Session() as sess:
    cl = tf.one_hot(label,10,axis=0)
    labels = sess.run(cl)

In [83]:
labels = labels.T

In [84]:
labels.shape

(42000, 10)

In [88]:
model.compile(optimizer=keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [90]:

model.fit(x=results,y=labels,epochs=5,batch_size=32)

Epoch 1/5
42000/42000 [==============================] - 2563s 61ms/step - loss: 0.2909 - acc: 0.9357
Epoch 2/5
42000/42000 [==============================] - 2433s 58ms/step - loss: 0.0883 - acc: 0.9747
Epoch 3/5
42000/42000 [==============================] - 2466s 59ms/step - loss: 0.2285 - acc: 0.9605
Epoch 4/5
42000/42000 [==============================] - 2486s 59ms/step - loss: 0.1609 - acc: 0.9655
Epoch 5/5
42000/42000 [==============================] - 2414s 57ms/step - loss: 0.0867 - acc: 0.9785


In [91]:
model.save('Res50.h5')

In [92]:
dfU = pd.read_csv("test.csv")

In [93]:
inputA = dfU.values

In [94]:
inputA.shape

(28000, 784)

In [95]:
X2 = []
for i in range(inputA.shape[0]):
    X2.append(inputA[i,:].reshape(28,28,1))

In [96]:
inputA = np.array(X2)

In [97]:

finRes = model.predict(inputA)

In [99]:

resF=[]
for i in range(28000):
    resF.append(np.argmax(finRes.T[:,i]))

In [100]:
answer = pd.DataFrame({"ImageId":range(1,28000+1),"Label":resF})

In [101]:
answer.to_csv("vote.csv",index=False)